In [28]:
import string
from itertools import groupby

class Plate():
    def __init__(self, height:int, length:int) -> None:
        positions = []
        for letter in list(string.ascii_uppercase[0:height]):
            for number in range(length):
                positions.append((letter, number+1))
        self.height = height
        self.length = length
        self.locations = [x[0]+str(x[1]) for x in positions]
    def rows(self)-> list:
        return [list(g) for k, g in groupby(self.locations, key=lambda x: x[0])]
    def columns(self) -> list:
        return [[x for x in self.locations if x[1:] == str(num+1)] for num in range(self.length)]
    def reverse_columns(self)-> list:
        return [sorted(x, reverse=True) for x in self.columns()]
        
standard96 = Plate(8,12)
standard384 = Plate(16,24)

class TipRack(Plate):
    def __init__(self,height:int=8,length:int=12)-> None:
        super(TipRack,self).__init__(height,length)
    
    def use_tips(self,tips:list)-> str:
        self.locations = [v for i, v in enumerate(self.locations) if v not in tips]
        return tips[-1]
    
    def use_number(self,num_tips:int=8) -> str:
        for column in tip.reverse_columns():
            if len(column) >= num_tips:
                return self.use_tips(column[0:num_tips])
        raise ValueError("No more tips.")
                
                

In [46]:
standard96.rows()[0][0:2]

['A1', 'A2']

In [8]:
tips.reverse_columns()

[['H1', 'G1', 'F1', 'E1', 'D1', 'C1', 'B1'],
 ['H2', 'G2', 'F2', 'E2', 'D2', 'C2', 'B2', 'A2'],
 ['H3', 'G3', 'F3', 'E3', 'D3', 'C3', 'B3', 'A3'],
 ['H4', 'G4', 'F4', 'E4', 'D4', 'C4', 'B4', 'A4'],
 ['H5', 'G5', 'F5', 'E5', 'D5', 'C5', 'B5', 'A5'],
 ['H6', 'G6', 'F6', 'E6', 'D6', 'C6', 'B6', 'A6'],
 ['H7', 'G7', 'F7', 'E7', 'D7', 'C7', 'B7', 'A7'],
 ['H8', 'G8', 'F8', 'E8', 'D8', 'C8', 'B8', 'A8'],
 ['H9', 'G9', 'F9', 'E9', 'D9', 'C9', 'B9', 'A9'],
 ['H10', 'G10', 'F10', 'E10', 'D10', 'C10', 'B10', 'A10'],
 ['H11', 'G11', 'F11', 'E11', 'D11', 'C11', 'B11', 'A11'],
 ['H12', 'G12', 'F12', 'E12', 'D12', 'C12', 'B12', 'A12']]

In [55]:
from opentrons import ProtocolContext as protocol_context
# metadata
metadata = {
    'protocolName': 'Test protocol',
    'author': 'Keoni Gandall <koeng101@gmail.com>',
    'description': 'Simple protocol to get started using OT2',
    'source': 'Opentrons Protocol Tutorial'
}

def run(protocol_context):
    
    # Establish tipracks
    tips_10 = TipRack()
    tips_300 = TipRack()
    tiprack_10 = protocol_context.load_labware_by_name('opentrons_96_tiprack_10_uL', '10')
    tiprack_300 = protocol_context.load_labware_by_name('opentrons_96_tiprack_300_uL', '11')

    # labware
    reagents = protocol_context.load_labware_by_name('generic_96_wellPlate_380_uL', '4')
    magdeck = protocol_context.load_module('Magnetic Module', '5')
    dna_output = magdeck.load_labware_by_name('generic_96_wellPlate_380_uL')
    tube_rack = protocol_context.load_labware_by_name('generic_96_wellPlate_380_uL', '6') # didn't want to figure out configuring tube racks, I do everything in plates
    final = protocol_context.load_module('generic_96_wellPlate_380_uL', '1')
    
    # cheat codes
    atm = reagents.columns[0]
    npm = reagents.columns[1]
    beads = reagents.columns[2]
    dna_target = dna_output.wells(standard96.rows()[0][0:2])
    
    i7_targets = tube_rack.wells('A3','A4')
    

    
    # pipettes
    pipette_10 = protocol_context.load_instrument('p10_multi', 'left', tip_racks=[tiprack_10])
    pipette_300 = protocol_context.load_instrument('p300_multi', 'right', tip_racks=[tiprack_300])

    
    # commands
    pipette_10.pick_up_tip().wells(tips_10.use_number(8))
    pipette_10.transfer(5,atm,dna_target,mix_after=(4,3),trash=True,new_tip='never')
    
    # Don't be evil, use a new tip for every index
    for index,i7 in enumerate(i7_targets):
        for well in standard96.columns()[index]:
            pipette_10.pick_up_tip().wells(tips_10.use_number(1))
            pipette_10.transfer(5,i7,well,trash=True,new_tip='never')
    # i5 time
    for column in standard96.rows()[0][0:2]:
        pipette_10.pick_up_tip().wells(tips_10.use_number(8))
        pipette_10.transfer(5,tube_rack.wells(column),dna_output.wells(column),trash=True,new_tip='never')
        
             
    pipette_10.pick_up_tip().wells(tips_10.use_number(8))
    pipette_10.transfer(15,npm,dna_target,mix_after=(4,3),trash=True,new_tip='never',touch_tip=True)
    
    pipette_300.pick_up_tip().wells(tips_300.use_number(8))
    pipette_300.transfer(30,beads,dna_target,mix_after=(20,30),trash=True,new_tip='never')
    
    protocol_context.delay(minutes=5)
    
    magdeck.engage()
    protocol_context.delay(minutes=2)
    
    for column in standard96.rows()[0][0:2]: 
        pipette_10.pick_up_tip().wells(tips_10.use_number(8))
        pipette_10.transfer(20,dna_output.wells(column).top(),final.wells(column),trash=True,new_tip='never')
    
    magdeck.disengage()

    

In [51]:
from opentrons import ProtocolContext as protocol_context